# Building train data:

In [1]:
import numpy as np
from random import randint
from math import floor, fabs
from cplex.callbacks import UserCutCallback
import cplex  as CPX
import cplex.callbacks as CPX_CB
import sys
import time
import pandas as pd
from itertools import product
import random
from sklearn import preprocessing
import copy

# Parameters:

In [2]:
examples=['data/NewExample1','data/NewExample1Redundant0.1','data/NewExample1Redundant0.25','data/NewExample1Redundant0.5']
# eps for class one
eps_one=0.001
# eps for class zero
eps_zero=0.001
# number of loop for generating class zero
LoopForZeros=[2]

# Building and saving the train data:

In [3]:
for LoopForZero in LoopForZeros:
    for example in examples:
        c = CPX.Cplex(example+'.lp')
        m=c.linear_constraints.get_num()
        n=c.variables.get_num()
        FinalCoeff=[]
        for i in range(m):
            my_row=[]
            for j in range(n):
                my_row.append(c.linear_constraints.get_coefficients(i,j))
            FinalCoeff.append(my_row) 
        FinalBounds=[]
        for i in range (n):
            tmp=[]
            for j in range (n):
                if i==j:
                    tmp.append(-1)
                else:
                    tmp.append(0)
            FinalBounds.append(tmp) 

        dfColumnName=[]  
        for i in range (n):
            dfColumnName.append("Var"+str(i))  
        dfColumnName.append("Output")           
        ######Train Data   
        trainingStartTime=time.time()
        FinalData=np.array([]).reshape(0, n+1)
        lenFinalCoeff=len(FinalCoeff)


        ##regular const class one with rand
        newrow=np.array(FinalCoeff)*-1
        normalized=preprocessing.normalize(newrow, axis=1, norm='l2')
        normalized = np.append(normalized,np.ones([len(normalized),1]),1)
        FinalData=np.append(FinalData,normalized, axis=0)
        for k in range (LoopForZero-1):
            #regular const class one with ones vector
            OnesVector=np.ones((lenFinalCoeff,n))
            newrow=np.array(FinalCoeff)+OnesVector*eps_one
            normalized=preprocessing.normalize(newrow, axis=1, norm='l2')
            normalized = np.append(normalized,np.ones([len(normalized),1]),1)
            FinalData=np.append(FinalData,normalized, axis=0)

        ##regular const class zero with rand
        for k in range (LoopForZero):
            PosRandNum=np.random.uniform(1,10000,(lenFinalCoeff,n)) 
            newrow=np.array(FinalCoeff)-PosRandNum*eps_zero
            normalized=preprocessing.normalize(newrow, axis=1, norm='l2')
            normalized = np.append(normalized,np.zeros([len(normalized),1]),1)
            FinalData=np.append(FinalData,normalized, axis=0)


        #bound constraint class one
        newrow=np.array(FinalBounds)*-1 
        newrow = np.append(newrow,np.ones([len(newrow),1]),1)
        FinalData=np.append(FinalData,newrow, axis=0)

        #bound constraint class one with ones vector
        for k in range (LoopForZero-1):
            OnesVector=np.ones((len(FinalBounds),n))
            newrow=np.array(FinalBounds)+OnesVector*eps_one 
            normalized=preprocessing.normalize(newrow, axis=1, norm='l2')
            normalized = np.append(normalized,np.ones([len(normalized),1]),1)
            FinalData=np.append(FinalData,normalized, axis=0)        

        #bound constraint class zero with rand 
        for k in range (LoopForZero):
            PosRandNum=np.random.uniform(1,10000,(len(FinalBounds),n)) 
            newrow=np.array(FinalBounds)-PosRandNum*eps_zero
            normalized=preprocessing.normalize(newrow, axis=1, norm='l2')
            normalized = np.append(normalized,np.zeros([len(normalized),1]),1)
            FinalData=np.append(FinalData,normalized, axis=0)

        df = pd.DataFrame(data=FinalData, columns=[dfColumnName])
        df.to_csv(example+'train.csv')      